<a href="https://colab.research.google.com/github/DavidSchineis/Math-Physics/blob/main/Copy_of_Lab_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract
This lab explores Fourier transforms and their applications to signals, images, and optics. We began by plotting 1D Fourier transforms of simple signals, and used FFT and IFFT to learn how frequency can be used to map back to the original function. We then moved on to 2D Fourier transforms to study point spread functions (PSFs) formed by different apertures. Circular and square apertures were compared to show how aperture size and shape directly effect the resolution and quality of real photos. Finally, we constructed a PSF using the real aperture mask of the James Webb Space Telescope and found the same PSF pattern seen in actual JWST images. This lab demonstrates how Fourier analysis is used in real world applications.

In [ ]:
import numpy as np
from numpy import pi, cos, sin
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from scipy.fft import fft2, fftshift
import requests
from PIL import Image
from io import BytesIO

Consider the signal represented by a function $$f=sin(2\pi\omega_1 t)+\frac{1}{2}sin(2\pi\omega_2 t)$$

First compute the fourier transform of this function by hand.

Then implement this function and make a plot of it as a function of time.

In [ ]:
t = np.linspace(-1,1,1001) # Uniformly spaced time axis ranging from -1 to 1
omega1 = 3  # Frequency of first sine wave
omega2 = 20  # Frequency of second sine wave
signal = sin(2*pi*omega1*t) + 1/2*sin(2*pi*omega2*t)

plt.plot(t, signal)
plt.show()


### Caption
This is a plot of the function
$$
f=sin(2\pi\omega_1 t)+\frac{1}{2}sin(2\pi\omega_2 t)
$$
where t goes from -1 to 1.

The function fft performs 1-D discrete Fourier transforms in Python, returning the magnitude of the signal at different frequencies. fftfreq, on the other hand, calibrates the internal scale of these frequencies through considering number of points and the spacing between them.

Make 3 plots: of the real part, the imaginary part of fft_signal, as well as the magnitude of it. Make sure to label all of the plots.

The returned array following fft would have the same length as the original array, so the bulk of it may not be of use. Zoom in on only the relevant portion of the figure to confirm your manual measurements with the automated calculation.

By default, frequencies are ordered starting from 0 to the end of all of the positive frequencies, then jumping to the most negative frequency, and continuiung on to end at 0. You can use fftshift function in plotting to reorder them (e.g., fftshift(freq), doing this to the value plotted on both x-axis and y-axis)

In [ ]:
fft_signal = fft(signal)

freq = fftfreq(len(signal), t[1] - t[0])

plt.title("Real Part")
plt.plot(freq, np.real(fft_signal))
plt.show()

plt.title("Imaginary Part")
plt.plot(freq, np.imag(fft_signal))
plt.show()

plt.title("Magnitude")
plt.plot(freq, np.abs(fft_signal))
plt.show()


### Caption
These are plots of the real, imaginary, and magnitude components of the 1D Fourier transform of the signal.

#### Question

Interpret the physical meaning of the Fourier transform with regards to the original function. What do different peaks and the power stored in them represent?

#### Answer
Each peak corresponds to a sin component from the signal, with the location of the peak representing its frequency and the height of the peak representing how much power the signal contains at that frequency.

----
Similarly, we can undo the transformation through using ifft function. Plot the real part, and the imaginary part of the reconstructed_signal as a function of t.

In [ ]:
reconstructed_signal = ifft(fft_signal)

plt.plot(t, reconstructed_signal.real)
plt.title("Real Part of Reconstructed Signal")
plt.show()

plt.plot(t, reconstructed_signal.imag)
plt.title("Imaginary Part of Reconstructed Signal")
plt.show()

### Caption
These are plots of the real and imaginary components of the inverse 1D Fourier transform of the signal. The real part seems to fully reconstruct the original signal on its own while the imaginary part remains near zero, because the original function was real.

Now let's consider a delta distribution. You can imitate the delta distribution through creating an array of zeros which has the same size as t, and setting the middle element to 1.

Repeat the exercise, plotting the function with respect to time, plotting real and imaginary parts of their Fourier transform as well as their magnitude their magnitude (keep all three can be on the same plot; make sure to include the legend with a label of all of the traces), and the inverse undoing the transformation.

You may want to use plt.scatter instead of plt.plot.

In [ ]:
delta = np.zeros_like(t)
delta[len(t)//4] = 1

plt.title("Delta Function")
plt.scatter(t, delta)
plt.show()


fft_signal = fft(delta)

freq = fftfreq(len(delta), t[1] - t[0])

plt.title("Real Part")
plt.scatter(freq, np.real(fft_signal))
plt.show()

plt.title("Imaginary Part")
plt.scatter(freq, np.imag(fft_signal))
plt.show()

plt.title("Magnitude")
plt.scatter(freq, np.abs(fft_signal))
plt.show()



reconstructed_signal = ifft(fft_signal)

plt.scatter(t, reconstructed_signal.real)
plt.title("Real Part of Reconstructed Delta")
plt.show()

plt.scatter(t, reconstructed_signal.imag)
plt.title("Imaginary Part of Reconstructed Delta")
plt.show()

### Caption
These are plots of the delta, its real component, its imaginary component, its reconstructed real component, and its reconstructed imaginary component of the inverse 1D Fourier transform of the delta. The real part seems to fully reconstruct the original signal on its own while the imaginary part remains near zero, because the original function was real.

#### Question
Instead of a delta function directly in the middle, try setting it at a different index. Compare the resulting Fourier transform. Qualitatively, how does changing the location of the delta function changes the Fourier transform? What remains the same in such a change?

#### Answer
Changing where the delta is only shifts the phase of each component, which changes the real and imaginary parts of the Fourier transform. The magnitude stays exactly the same, since delta always has all of the other frequencies equal zero.

----
Fourier transforms have a number of physical applications. For example, in optics, it is a natural outcome of imaging light through a finite sized aperture. The resulting point spread function (PSF) that defines how the light would be distributed on the detector relates to a 2-dimentional Fourier transform over the aperture of your optical system.

Consider two telescopes, one with the diameter of 6 pixels, the other with a diameter of 60 pixels.

In [ ]:
def make_circular_aperture(aperture_size=6,grid_size=101):
    aperture = np.zeros((grid_size, grid_size))

    for i in range(grid_size):
        for j in range(grid_size):
            x = (i - grid_size / 2)
            y = (j - grid_size / 2)
            if np.sqrt(x**2 + y**2) <= aperture_size / 2:
                aperture[i, j] = 1

    return aperture


fig, ax= plt.subplots(1,2)
aperture1=make_circular_aperture(aperture_size=6)
aperture2=make_circular_aperture(aperture_size=60)
ax[0].imshow(aperture1)
ax[1].imshow(aperture2)
plt.show()

This is the resulting PSF of one of these telescopes. Compare it with the PSF of the other one.

In [ ]:
fft_aperture1 = fftshift(fft2(aperture1))
psf1 = np.abs(fft_aperture1)**2

plt.figure()
plt.imshow(psf1/np.sum(psf1), cmap='hot',vmin=1e-6,vmax=5e-5)
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.title('Point Spread Function 1')
plt.colorbar()
plt.show()

#second PSF
fft_aperture2 = fftshift(fft2(aperture2))
psf2 = np.abs(fft_aperture2)**2

plt.figure()
plt.imshow(psf2/np.sum(psf2), cmap='hot',vmin=1e-6,vmax=5e-5)
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.title('Point Spread Function 2')
plt.colorbar()
plt.show()

### Caption
These are two plots comparing the point spread function of different sized circular aperatures. The first plot is in reference to a 6 pixel diameter while the second is in reference to a 60 pixel diameter.

#### Question

How does the size of the PSF depend on the telescope size? From this, what can you infer about the image quality of an image obtained through these optics?

#### Answer
The PSF decreases as telescope size increases. Since the PSF shows the amount of light being passed onto the detector, I would assume that the closer focus around the second, larger aperature would result in better image quality.

----
Now make a square aperture.

Hint: there are a copule of ways of doing it. If using two nested for loops as above, consider condition that should go into the if statement inside of it. You can also do entirely without for loops in such a case and just specify a list of indexes where the value corresponding to the aperture should be 1.

In [ ]:
def make_square_aperture(aperture_size=6,grid_size=101):
    aperture = np.zeros((grid_size, grid_size))
    center = grid_size/2

    for i in range(grid_size):
        for j in range(grid_size):
            x = i - center
            y = j - center

            if (abs(x) <= aperture_size/2) and (abs(y) <= aperture_size/2):
                aperture[i, j] = 1

    return aperture


aperture1=make_square_aperture(aperture_size=6)
aperture2=make_square_aperture(aperture_size=60)

fft_aperture1 = fftshift(fft2(aperture1))
psf1 = np.abs(fft_aperture1)**2

plt.figure()
plt.imshow(psf1/np.sum(psf1), cmap='hot',vmin=1e-6,vmax=5e-5)
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.title('Point Spread Function 1')
plt.colorbar()
plt.show()

#second PSF
fft_aperture2 = fftshift(fft2(aperture2))
psf2 = np.abs(fft_aperture2)**2

plt.figure()
plt.imshow(psf2/np.sum(psf2), cmap='hot',vmin=1e-6,vmax=5e-5)
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.title('Point Spread Function 2')
plt.colorbar()
plt.show()

### Caption
These are two plots comparing the point spread function of different sized square aperatures. The first plot is in reference to a 6 pixel diameter while the second is in reference to a 60 pixel diameter.

#### Question
How is the shape of the PSF affected by the aperture shape?

#### Answer
The shape of the PSF directly follows the aperture shape. A circular aperture produces a circular PSF, while a square aperture produces a square, cross shaped PSF.

----
Finally, let's consider a more complex system. In images produced by the James Webb Space telescope, one of the distinguishing features is six pointed diffraction spikes around all of the stars. JWST has a very complex aperture that is troublesome to recreate from scratch, so instead we will use the image of its optics as a basis.

In [ ]:
# URL of the image
image_url = "https://mkounkel.domains.unf.edu/MP/jwst-selfie.jpeg"



headers ={

 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'

}

# Download the image

response = requests.get(image_url,stream=True,headers=headers)

image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = image.convert("L")

# Convert the grayscale image to a NumPy array
aperture = np.array(gray_image)

# Set all grey pixels to white
aperture[aperture > 100] = 255
aperture[aperture <= 100] = 0

plt.imshow(aperture)

Create a PSF associated with this aperture. Since there is little power in the diffraction spikes, pass arguments "vmin=1e-8,vmax=5e-6" to imshow to see them. Compare the resulting image with some images from JWST.

In [ ]:
# Compute PSF
fft_aperture = fftshift(fft2(aperture))
psf = np.abs(fft_aperture)**2

plt.imshow(psf/np.sum(psf), cmap='hot', vmin=1e-8, vmax=5e-6)
plt.title("JWST PSF")
plt.colorbar()
plt.show()


### Caption
This is a plot of the PSF from the JWST aperture image. There are six distinct spikes produced in the PSF. These resemble those seen in real JWST images, following how the telescopes mirror geometry directly shapes its pattern.

### Extra Credit

Cosmic Microwave Background is an (almost) uniform emission seen across the entire sky. Corresponding to a temperature of 2.726 K, it is a remnant of the earliest moments of the Big Bang. Number of missions, such as Planck, have imaged it.

This dataset corresponds to the temperature fluctuations from 2.726 K across the sky. Since the sky is spherical, and the dataset is somewhat specialized, special tools are used to read and interpret these data.
There is also a specialized function to take a power spectrum (square the magnitude of the Fourier transform) of this dataset.

In [ ]:
!pip install healpy
import healpy as hp
cmb_map = hp.read_map('https://irsa.ipac.caltech.edu/data/Planck/release_2/all-sky-maps/maps/component-maps/cmb/COM_CMB_IQU-commander_1024_R2.02_full.fits')
hp.mollview(cmb_map, min=-3e-4, max=3e-4, title="CMB only temperature map", unit="K")
plt.show()


lmax = 3000
test_cls_meas_frommap = hp.anafast(cmb_map, lmax=lmax, use_pixel_weights=True)
ll = np.arange(lmax+1)
k2muK = 1e6
def multipole2ang(x):
    return 180./x
def ang2multipole(x):
    return 180./x
fig, ax = plt.subplots(layout='constrained')
plt.plot(ll, ll*(ll+1.)*test_cls_meas_frommap*k2muK**2/2./np.pi, '--', alpha=0.6, label='Planck 2018 PS from Data Map')
plt.xlabel(r'Multipole moment')
plt.ylabel(r'$D_\ell~[\mu K^2]$')
plt.xlim(0.1,1000)
secax = ax.secondary_xaxis('top', functions=(multipole2ang, ang2multipole))
secax.set_xlabel(r'Angular scale (deg)')
secax.set_xticks(np.array([0.2,0.3,0.4,0.5,1,2,5]))

plt.grid()
plt.legend(loc='best')
plt.show()

#### Question

Interpret the meaning of the location of the first peak of the CMB power spectrum solely as it pertains to the all-sky image (i.e., what does it correspond to in the image, without considering astrophysical significance)

#### Answer